#Estimating the Test Error

We learned about _training error_, which is the error calculated on the training data. Training error is easy to calculate because the true labels are available in the training data. However, training error is not always a good measure of a model's quality, since a model that _overfits_ to the training data may have artificially low training error.

Ideally, we would like to evaluate regression models based on their _test error_, which is the error calculated on the test data. The problem with test error is that it is usually impossible to calculate, since the true labels are rarely available in the test data. In this section, we discuss strategies for estimating the test error using only the training data.

In [1]:
import pandas as pd
import numpy as np

# Extract the training data.

bordeaux_df = pd.read_csv("bordeaux.csv",
                          index_col="year")
bordeaux_train = bordeaux_df.loc[:1980].copy()
bordeaux_train["log(price)"] = np.log(bordeaux_train["price"])

## Validation Error

To estimate the test error, we split the training data into a _training set_ and a _validation set_. First, the model is fit to just the data in the training set. Then, the model is evaluated based on its predictions on the validation set. Because the model did not train on any of the labels in the validation set, the validation set essentially plays the role of the test data, even though it was carved out of the training data.

The prediction error on the validation set is known as the _validation error_. The validation error is an approximation to the test error.

To split our data into training and validation sets, we can use the `.sample()` function in `pandas`. Let's use this to split our data into two equal halves, which we will call `train` and `val`.

In [2]:
train = bordeaux_train.sample(frac=.5)
val = bordeaux_train.drop(train.index)

train

,price,summer,har,sep,win,age,log(price)
year,,,,,,,
1980,14.0,16.0,74,18.4,578,12,2.639057
1957,22.0,16.1,110,16.2,420,35,3.091042
1952,37.0,17.1,160,14.3,600,40,3.610918
1964,31.0,17.3,96,18.8,402,28,3.433987
1979,21.0,16.2,122,17.3,717,13,3.044522
1973,16.0,17.1,123,17.9,376,19,2.772589
1959,66.0,17.5,187,18.7,485,33,4.189655
1961,100.0,17.3,38,20.4,830,31,4.605170
1966,47.0,16.5,86,18.4,819,26,3.850148


Now let's use these training/validation sets to approximate the test MSE of the 5-nearest neighbors model that we fit in Chapter 5.2.

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline

# extract features and label from training set
X_train = train[["win", "summer"]]
y_train = train["log(price)"]

x_train.

# define pipeline and fit to training set
pipeline = make_pipeline(
          StandardScaler(),
          KNeighborsRegressor(n_neighbors=5)
)
pipeline.fit(X=X_train, y=y_train)

SyntaxError: invalid syntax (127438805.py, line 9)

We make predictions on the validation set and calculate the validation RMSE:

In [ ]:
from sklearn.metrics import mean_squared_error

# extract features and label from validation set
X_val = val[["win", "summer"]]
y_val = val["log(price)"]

# get model's predictions on validation set
y_val_ = pipeline.predict(X_val)

# calculate RMSE on validation set
rmse = np.sqrt(mean_squared_error(y_val, y_val_))
rmse

0.5934339137629878

Notice that the test error is higher than the training error that we calculated in the previous lesson. In general, this will be true. It is harder for a model to predict for new observations it has not seen, than for observations it has seen!

## Cross Validation

The validation error above was calculated using only 50% of the training data, since we split the training data in half to create the validation set. As a result, the estimate is noisy.

There is a cheap way to obtain a second opinion about how well our model will do on future data. Previously, we split our data at random into two halves, fitting the model to the first half and evaluating it on the second half. Because the model has not already seen the second half of the data, this approximates how well the model would perform on future data. 

But the way we split our data was arbitrary. We might as well swap the roles of the two halves, fitting the model to the _second_ half and evaluating it on the _first_ half. As long as the model is always evaluated on data that is different from the data that was used to train it, we have a valid estimate of test error. A schematic of this approach, known as _cross-validation_, is shown below.

![](https://github.com/dlsun/pods/blob/master/05-Regression-Models/cross-validation.png?raw=1)

Because we will be doing all computations twice, just with different data, let's wrap the $k$-nearest neighbors algorithm above into a function called `get_val_error()`, that computes the validation error given training and validation data.

In [ ]:
def get_val_error(train, val):
    
    # extract features and label from training set.
    X_train = train[["win", "summer"]]
    y_train = train["log(price)"]
    
    # eefine pipeline and fit to training set
    pipeline = make_pipeline(
          StandardScaler(),
          KNeighborsRegressor(n_neighbors=5)
    )
    pipeline.fit(X=X_train, y=y_train)
    
    # extract features and label from validation set
    X_val = val[["win", "summer"]]
    y_val = val["log(price)"]
    
    # get model's predictions on validation set
    y_val_ = pipeline.predict(X_val)

    # calculate RMSE on validation set
    rmse = np.sqrt(mean_squared_error(y_val, y_val_))
    
    return rmse

If we apply this function to the training and validation sets from earlier, we get the same estimate of the test error.

In [ ]:
get_val_error(train, val)

0.5934339137629878

But if we reverse the roles of the training and validation sets, we get another estimate of the test error.

In [ ]:
get_val_error(val, train)

0.6989515061348369

Now we have two, somewhat independent estimates of the test error. It is common to average the two numbers to obtain an overall estimate of the test error, called the _cross-validation estimate of test error_. Notice that the cross-validation estimate uses each observation in the data exactly once. We make a prediction for each observation, but always using a model that was trained on data that does not include that observation.

## Cross-Validation in scikit-learn

As you know by now, scikit-learn provides functions that automate routine tasks of machine learning. For cross-validation, there is a function, `cross_val_score`, that takes in a model (or pipeline), the training data, and a scoring function, and carries out all aspects of cross-validation, including

1. splitting the training data into training and validation sets
2. fitting the model to each training set
3. calculating the model's predictions on the corresponding validation set
4. calculating the score of the predictions on each validation set.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(pipeline, 
                         X=bordeaux_train[["win", "summer"]],
                         y=bordeaux_train["log(price)"],
                         scoring="neg_mean_squared_error",
                         cv=2)
scores

array([-0.47767636, -0.38655855])

First, notice that there are 2 scores. This is because scikit-learn calculated a score from each half of the data when that half served as the validation set.

Second, observe that the scores are negative. This is because scikit-learn requires that a "score" be something that ought to be maximized. Since we want to minimize the mean-squared error, we want to maximize the *negative* mean-squared error. Therefore, the scores that are reported here are the negative of the MSE.

To calculate the RMSE, we negate the negative sign and take a square root.

In [ ]:
np.sqrt(-scores)

array([0.69114134, 0.62173833])

The average of these two scores is the cross-validation estimate of test error.

In [ ]:
np.sqrt(-scores).mean()

0.6564398373317621

## $K$-Fold Cross Validation

One problem with splitting the training data into two halves is that the model is now trained on only half the amount of data. This model will likely perform worse than the actual model, which is trained on all of the training data. So the cross-validation estimate of test error is unnecessarily pessimistic.

We would like the size of the training set to be closer to the size of the original training data. We can do this by splitting the data into more than two subsamples. In general, we can split the data into $k$ subsamples, alternately training the data on $k-1$ subsamples and evaluating the model on the $1$ remaining subsample, i.e., the validation set. This produces $k$ somewhat independent estimates of the test error. This procedure is known as **$k$-fold cross validation**. (Be careful not to confuse this $k$ with the $k$ in $k$-nearest neighbors.) In hindsight, the  cross validation that we were doing previously is $2$-fold cross validation.

A schematic of $4$-fold cross validation is shown below.

![](https://github.com/dlsun/pods/blob/master/05-Regression-Models/k-folds.png?raw=1)

Implementing $k$-fold cross validation in scikit-learn is easy. We simply set the `cv=` parameter to the desired number of folds. For example, the following code carries out $4$-fold cross validation.

In [ ]:
scores = cross_val_score(pipeline, 
                         X=bordeaux_train[["win", "summer"]],
                         y=bordeaux_train["log(price)"],
                         scoring="neg_mean_squared_error",
                         cv=4)
scores

array([-0.47213584, -0.30601689, -0.36874833, -0.11290777])

Notice that $k$ scores are produced, one from each fold. These scores can be averaged to produce a single estimate of the test error.

In [ ]:
np.sqrt(-scores).mean()

0.545893343925109